<a href="https://colab.research.google.com/github/Camipinto/Laboratorio-de-Programacion-Cientifica-para-Ciencia-de-Datos/blob/main/Lab06/Lab06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: **Camila Pinto L.**
- Nombre de alumno 2: **Camilo Ramírez C.**


### **Link de repositorio de GitHub:** https://github.com/Camilormz/MDS7202-labs

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 09/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
# try:
#     from google.colab import drive
#     drive.mount("/content/drive")
#     path = 'Dirección donde tiene los archivos en el Drive'
# except: 
print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [ ]:
%%capture
# Descarga directa de los datos desde Internet
!wget -O online_retail_II_cleaned.pickle https://drive.google.com/u/0/uc?id=1K58X_3AtkZxp1sbMKD7ZPM55_HJw9yVB&export=download

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [ ]:
from pandas.core.frame import DataFrame as PandasDF

df_retail: PandasDF = pd.read_pickle(
    filepath_or_buffer="online_retail_II_cleaned.pickle"
)
df_retail = df_retail.astype({
    "Invoice": "category", "StockCode": "category", "Description": "category",
    "Description": str, "Customer ID": "category", "Country": "category"
})
df_retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
from pandas.core.groupby.generic import DataFrameGroupBy as PandasGroupDF
from pandas.core.groupby.generic import SeriesGroupBy as PandasGroupSeries
from pandas.core.series import Series as PandasSeries

def custom_features(dataframe_in: PandasDF) -> PandasDF:
    """
    Calcula un dataframe de características a partir de un dataframe de datos

    Parameters
    ----------
    dataframe_in
        Dataframe de datos de entrada
    
    Returns
    -------
    Dataframe de características
    """
    client_group: PandasGroupDF = dataframe_in.copy().groupby(["Customer ID"])
    invoices_client_group: PandasGroupSeries = client_group["InvoiceDate"]

    # Cálculo de length
    max_invoicedate_per_client: PandasSeries = invoices_client_group.max()
    length_series: PandasSeries = (
        max_invoicedate_per_client - invoices_client_group.min()
    ).apply(lambda x: x.days).rename("Length")

    # Cálculo de recency
    final_date: np.datetime64 = (
        max_invoicedate_per_client.max() + pd.Timedelta(value=1, unit='D')
    ).to_datetime64()
    unique_invoices_per_client: PandasSeries = invoices_client_group.unique()
    recency_series: PandasSeries = unique_invoices_per_client.apply(
        lambda x: np.mean(final_date - x).astype("timedelta64[D]")
    ).rename("Recency")

    # Cálculo de frequency
    frequency_series: PandasSeries = client_group["Invoice"].nunique().rename(
        "Frequency"
    )

    # Cálculo de monetary
    monetary_series: PandasSeries = (
        client_group["Price"].sum() / frequency_series
    ).rename("Monetary")

    # Cálculo de periodicity
    periodicity_series: PandasSeries = unique_invoices_per_client.apply(
        lambda x: np.std(np.diff(x).astype("timedelta64[D]").astype(np.int64))
    ).fillna(0).rename("Periodicity")

    # Construcción y retorno del dataframe de features
    return pd.concat(
         [length_series, recency_series, frequency_series, monetary_series,
          periodicity_series], axis=1
    )

**Resúesta de Pauta:**

In [ ]:
df_features: PandasDF = custom_features(df_retail)
df_features

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning:

Degrees of freedom <= 0 for slice



,Length,Recency,Frequency,Monetary,Periodicity
Customer ID,,,,,
12346.0,196,325 days,11,18.7600,34.778729
12347.0,37,21 days,2,81.4750,0.000000
12348.0,0,74 days,1,14.3900,0.000000
12349.0,181,158 days,3,291.7800,72.000000
12351.0,0,11 days,1,49.4600,0.000000
...,...,...,...,...,...
18283.0,275,164 days,6,79.5450,55.304973
18284.0,0,67 days,1,91.0900,0.000000
18285.0,0,296 days,1,100.2000,0.000000


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [ ]:
from numpy.typing import ArrayLike
from typing import Optional, Any

class MinMax(BaseEstimator, TransformerMixin):
    """
    MinMax custom transformer class
    """
    def fit(self, X: ArrayLike, y: Optional[ArrayLike] = None):
        self.min = X.min()
        self.max = X.max()
        return self
    def transform(self, X: ArrayLike, y: Optional[ArrayLike] = None):
        return (X - self.min)/(self.max - self.min)

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE

pipeline: Pipeline = Pipeline(
    steps=[
        ("feature_extraction", FunctionTransformer(func=custom_features)),
        ("min_max_scaling", ColumnTransformer(transformers=[
            ("min_max_columns_scaling", MinMax(), df_features.columns)
            ])
        ),
        ("tsne", TSNE(n_components=2, random_state=1234))]
)

In [ ]:
tsne_output: np.ndarray = pipeline.fit_transform(df_retail)

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning:

Degrees of freedom <= 0 for slice

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [ ]:
px.scatter(x=tsne_output[:, 0], y=tsne_output[:, 1],
           title="T-SNE applied over retail features")

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [ ]:
%%capture
from sklearn.cluster import KMeans
from typing import List

pipelines: List[Pipeline] = [Pipeline(
    steps=[
        ("feature_extraction", FunctionTransformer(func=custom_features)),
        ("min_max_scaling", ColumnTransformer(transformers=[
            ("min_max_columns_scaling", MinMax(), df_features.columns)
            ])
        ),
        ("k-means", KMeans(n_clusters=k, random_state=1234))]
) for k in range(1, 21)]

k_means_pipelines: List[KMeans] = list()
for k in range(20):
    k_means_pipelines.append(pipelines[k].fit(df_retail))

In [ ]:
inertias: List[float] = [
    k_means_pipelines[k]["k-means"].inertia_ for k in range(20)
]
px.line(x=range(1, 21), y=inertias, title="Método del codo con k-means",
        labels={"x": "Número de clusters", "y": "Cluster inertia"})

A partir del gráfico del método del codo se decide escoger 3 clusters como la cantidad óptima de grupos para realizar el k-means.

Se cree que con 3 *clusters* se tiene un buen trade-off respecto a minimizar la distancia entre los datos y el centro de cada cluster y la complejidad del modelo. Ya que en este punto la inertia deja de disminuir abruptamente.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [ ]:
best_pipeline: Pipeline = k_means_pipelines[2]
pd.DataFrame(
    best_pipeline["k-means"].cluster_centers_,
    columns=["Length", "Recency", "Frequency", "Monetary", "Periodicity"]
).rename_axis("Cluster")

,Length,Recency,Frequency,Monetary,Periodicity
Cluster,,,,,
0,0.736765,0.461755,0.034262,0.006264,0.200781
1,0.089810,0.180345,0.004924,0.006794,0.019927
2,0.078813,0.704408,0.003521,0.007504,0.015431


A partir de los clusters formados es posible observar grupos coherentes ya que se ve diferencia significativa en las características del elemento representativo de cada cual.

Por ejemplo el primer grupo son los clientes más fieles y son el aquellos que visita la tienda con más regularidad.

El segundo grupo son aquellos clientes que se caracterizan por haber tenido mayor interacción actualmente con la tienda.

Y el tercer grupo de clientes son aquellos que son menos fieles, los que menos han visitado la tienda, lo hacen con menos regularidad  y han tenido menos interacción actualmente con la tienda

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [ ]:
px.scatter(x=tsne_output[:, 0], y=tsne_output[:, 1],
           color=best_pipeline["k-means"].labels_,
           title="T-SNE applied over retail features with 3 clusters")

En el gráfico se puede ver como los 3 grupos segmentados con k-means están en regiones separadas y practicamente no hay traslape. Solo unos pocos clientes del grupo 2 se mezclan con los otros grupos, sin embargo en términos generales los clusters separan bien a los distintos tipos de clientes.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>